<a href="https://colab.research.google.com/github/hieubkset/Colab-Notebooks/blob/master/text-classification-with-hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phân loại văn bản**

Một ví dụ về bài toán phân loại văn bản trên tập dữ liệu [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb).

Mục tiêu:

+ Có cái nhìn tổng quan về [TensorFlow Hub](https://www.tensorflow.org/hub) và [TensorFlow datasets](https://www.tensorflow.org/datasets).
+ Hiểu cơ bản việc sử dụng transfer learning để chuyển đổi từ text sang embedding vector.

## **Giới thiệu IMDB dataset**

[IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) bao gồm review (đánh giá) của 50,000 bộ phim từ [Internet Movie Database](https://www.imdb.com/). Trong đó, 25,000 review cho training và 25,000 reivew còn lại cho testing. Mỗi review là một câu bình luận về bộ phim và được gán một trong 2 nhãn: positive (tích cực) hoặc negative (tiêu cực). Số lượng nhãn positive và negative là chia đều trong cả tập training và testing. 

## **Chương trình**

### **1. Khai báo các thư viện**

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE" )

### **2. Chuẩn bị data**

Ở các bài trước, chúng ta đã học cách load data với **tf.keras.datasets**. Ở bài này, chúng ta sẽ làm quen với một kho data khác to hơn là [Tensorflow Datasets](https://www.tensorflow.org/datasets). 

**Tensorflow Datasets** cung cấp khoảng 29 bộ dataset như: MNIST, Street View House Numbers, the 1 Billion Word Language Model Benchmark, v.v... và được cập nhật thường xuyên.

Tất cả các dataset trong **Tensorflow Datasets** là một thể hiển của  [tf.data.Datasets](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Hiểu một cách đơn giản là các dataset này đã chuẩn hóa nên rất dễ sử dụng và cho hiệu năng cao.

Dưới đây là một ví dụ load **IMDB dataset** với **Tensorflow Datasets** (tfds):

In [0]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

** Explore data**

Chúng ta sẽ xem định dạng của dữ liệu trước khi tiến hành build model:

In [0]:
train_examples_batch, train_lables_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [0]:
train_lables_batch

Label gồm 2 giá trị: 1 tương ứng với positive và 0 tương ứng với negative.

### **3. Build model**

Để xử lý dữ liệu đầu vào là các review dưới dạng text có chiều dài bất kỳ, chúng ta sẽ sử dụng một pre-trained text embedding model để chuyển đầu vào từ text sang embedding vector (có chiều dài cố định).

Cụ thể, chúng ta sử dụng [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) từ [TensorFlow Hub](https://www.tensorflow.org/hub).

**TensorFlow Hub** là nơi cung cấp các pre-trained model - các model được train trên các tập dataset (thường nhiều data). Chúng ta có thể dùng toàn bộ hoặc một phần của pre-trained model để tiến hành training trên dataset của chúng ta (thường ít data). Quá trình này gọi là transfer learning.

**TensorFlow Hub** hỗ trợ việc sử dụng pre-trained model như một keras layer. Ví dụ: 


In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

*Chúng ta có thể thấy thấy dữ liệu được chuyển từ text sang embedding vector có 20 dim (chiều).*

**Định nghĩ model:**
+ pre-trained model được sử dụng như input layer chuyển đổi đầu vào từ text sang embedding vector có số chiều cố định là 20.
+ Dense layer với 16 hidden unit
+ Output layer với 1 unit. Vì bài toán chỉ có 2 class nên ta chỉ cần một output cho biết xác suất ứng với possitive.

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

#### **4. Khai báo Optimizer và Loss function**
Chúng ta vẫn sử dụng Adam Optimizer :) Tuy nhiên, vì chỉ có 2 class nên ta dùng **BinaryCrossentropy**.

In [0]:
model.compile(optimizer='adam', 
              loss= tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

### **5. Tiến hành training**

Train model trong 20 epoch với batch size là 512. Trong quá trình training, các thông số như loss và accuracy trên tập validation sẽ được lưu lại (*chúng ta sẽ tìm hiểu cách hiển thị các thông số này ở bài sau*).


In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

**6. Testing**

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

Với 20 epoch, model cho accuracy trên tập test khoảng 87%. Với cách tiếp cận nâng cao hơn, accuracy có thể lên tới 95%.